In [39]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch
from torch import nn, optim
import pytorch_lightning as pl
from preprocess import LitChartBioData


In [40]:
class DeepChartClassifier(pl.LightningModule):
    """
    This class performs chart classification with a deep neural network. 
    Very similar to the Logistic Regression model in baseline, but with 
    a two layer neural net: ultimately outputs a 0 or 1.
    """
    def __init__(self, num_features, num_hidden = 12):
        super().__init__()
        self.neural = nn.Sequential(
            nn.Linear(num_features, num_hidden),
            nn.ReLU(),
            nn.Linear(num_hidden, 1),
            nn.Sigmoid()
        )
    
    def training_step(self, batch, batch_idx):
        x, y = batch
        probs = self.neural(x.to(torch.float32))
        y_hat = (probs > 0.5).float()    # turn sigmoid outputs into 0s and 1s
        loss_fn = nn.BCELoss()
        loss = loss_fn(y_hat, y.to(torch.float))
        self.log('training_loss', loss)
        return loss
    
    def configure_optimizers(self):
        optimizer = optim.Adam(self.parameters())
        return optimizer


In [42]:
def run_and_plot_NN(num_hidden):
    """
    Run the two layer neural net for classification with
    passed in number of hidden neurons. Plot the training
    loss and test accuracy.
    
    Args:
        (num_hidden) number of hidden nodes
    """
    data_module = LitChartBioData(train_filepath='train_features_len_1024_tensor.pt', test_filepath='test_features_len_1024_tensor.pt')
    trainer = pl.Trainer()
    model = DeepChartClassifier(data_module.num_features, num_hidden)
    trainer.fit(model, data_module)
    return model, data_module


In [43]:
model, dm = run_and_plot_NN(12)

2022-12-10 10:06:09,487 - pytorch_lightning.utilities.rank_zero - INFO - GPU available: False, used: False
2022-12-10 10:06:09,500 - pytorch_lightning.utilities.rank_zero - INFO - TPU available: False, using: 0 TPU cores
2022-12-10 10:06:09,501 - pytorch_lightning.utilities.rank_zero - INFO - IPU available: False, using: 0 IPUs
2022-12-10 10:06:09,502 - pytorch_lightning.utilities.rank_zero - INFO - HPU available: False, using: 0 HPUs
/Users/Armaan/opt/miniconda3/envs/finalproj/lib/python3.10/site-packages/pytorch_lightning/loops/utilities.py:89: PossibleUserWarning: `max_epochs` was not set. Setting it to 1000 epochs. To train without an epoch limit, set `max_epochs=-1`.
  rank_zero_warn(
2022-12-10 10:06:09,539 - pytorch_lightning.loggers.tensorboard - WARNING - Missing logger folder: /Users/Armaan/Desktop/CS229/PROJECT/code/lightning_logs
2022-12-10 10:06:09,625 - pytorch_lightning.callbacks.model_summary - INFO - 
  | Name   | Type       | Params
-----------------------------------

Epoch 0:   0%|          | 0/115 [00:00<?, ?it/s] 

RuntimeError: element 0 of tensors does not require grad and does not have a grad_fn